# The Product Pricer Fine-Tuning a Frontier Model - Similation (GPT-4 mini)

Submitted By: Bharat Puri

A model that can estimate how much something costs, from its description.


In [14]:
# imports

import os
import re
import math
import json
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from collections import Counter
import sys
sys.path.append(os.path.abspath(os.path.join("..", ".."))) 
from openai import OpenAI
from anthropic import Anthropic
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


In [15]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# =============================================================
# Step 1 — Data Curation and Preparation (Integrated from 09_part1_data_curation)
# =============================================================

import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

print("🔍 Starting data curation...")

# Load input/output CSVs (adjust paths as needed)
df_input = pd.read_csv("../../human_input.csv")
df_output = pd.read_csv("../../human_output.csv")

# Detect and combine dynamically
i_col, o_col = df_input.columns[0], df_output.columns[0]
df = pd.DataFrame({
    "prompt": df_input[i_col].astype(str).str.strip(),
    "completion": df_output[o_col].astype(str).str.strip()
})

# Basic cleaning
df.dropna(inplace=True)
df = df[df["prompt"].str.len() > 0]
df = df[df["completion"].str.len() > 0]
df = df.reset_index(drop=True)

print(f"✅ Cleaned dataset shape: {df.shape}")
print(df.head(3))

# Split into training and validation
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}")

# Save curated datasets to reuse later
with open("train.pkl", "wb") as f:
    pickle.dump(train_df, f)
with open("test.pkl", "wb") as f:
    pickle.dump(val_df, f)

print("💾 Saved train.pkl and test.pkl successfully.")



In [ ]:
# =============================================================
# Step 2 — Prepare Data for Fine-Tuning
# =============================================================
import pickle
import pandas as pd

print("📦 Loading curated train/test data from pickle files...")

with open("train.pkl", "rb") as f:
    train_df = pickle.load(f)
with open("test.pkl", "rb") as f:
    val_df = pickle.load(f)

print(f"✅ Loaded train={len(train_df)} | val={len(val_df)}")

# Ensure correct column names
train_df = train_df.rename(columns={train_df.columns[0]: "prompt", train_df.columns[1]: "completion"})
val_df = val_df.rename(columns={val_df.columns[0]: "prompt", val_df.columns[1]: "completion"})

# Save as JSONL for OpenAI Fine-Tuning
train_df.to_json("train.jsonl", orient="records", lines=True)
val_df.to_json("val.jsonl", orient="records", lines=True)

print("💾 Saved train.jsonl and val.jsonl for fine-tuning.")

In [ ]:
# =============================================================
# Step 3 — Fine-Tuning Configuration
# =============================================================
import json

hyperparams = {
    "model": "gpt-4o-mini",          # Frontier model from the course
    "n_epochs": 3,                   # Small safe run
    "batch_size": 8,                 # Reasonable for small data
    "learning_rate_multiplier": 0.5, # Trainer's suggested mid value
    "suffix": "week6_bharat_ft_v1"   # Unique identifier for your run
}

print("⚙️ Fine-tuning configuration:")
print(json.dumps(hyperparams, indent=2))


In [ ]:
# =============================================
# Step 3 – Define Fine-Tuning Configuration
# =============================================

hyperparams = {
    "model": "gpt-4o-mini",            
    "n_epochs": 1,                     
    "batch_size": 4,                   # Small batch = less token use
    "learning_rate_multiplier": 0.5,   # Gentle learning rate
    "suffix": "week6_lowcost_bharat"   # Custom suffix for tracking
}

print("✅ Fine-tuning configuration defined:")
for k, v in hyperparams.items():
    print(f"{k:25}: {v}")


In [ ]:
# =============================================================
# Step 4 — Launch Fine-Tuning Job (Fixed for latest SDK)
# =============================================================
from openai import OpenAI
import time, os, json

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

simulate = True  # Set True for simulation (no cost)

if simulate:
    print("\n🧪 Simulation mode — running mock fine-tuning steps...")
    for e in range(3):
        print(f"Simulated Epoch {e+1}/3")
        time.sleep(1)
    ft_model = "ft:gpt-4o-mini:SIMULATED"
    print("✅ Simulation complete — no API cost.")
else:
    print("\n🚀 Creating fine-tuning job...")

    # Upload training and validation data
    train_file = client.files.create(file=open("train.jsonl", "rb"), purpose="fine-tune")
    val_file   = client.files.create(file=open("val.jsonl", "rb"),   purpose="fine-tune")

    # ✅ Correct usage: hyperparameters must go inside a dictionary named `hyperparameters`
    job = client.fine_tuning.jobs.create(
        model="gpt-4o-mini",
        training_file=train_file.id,
        validation_file=val_file.id,
        hyperparameters={
            "n_epochs": 3,
            "batch_size": 8,
            "learning_rate_multiplier": 0.5
        },
        suffix="week6_bharat_ft_v1"
    )

    print("🆔 Job created:", job.id)

    # Poll until completion
    status = job.status
    while status in ("validating_files", "queued", "running"):
        print("⏳ Status:", status)
        time.sleep(20)
        job = client.fine_tuning.jobs.retrieve(job.id)
        status = job.status

    if job.status != "succeeded":
        raise RuntimeError(f"❌ Fine-tune failed with status: {job.status}")

    ft_model = job.fine_tuned_model
    print("🎯 Fine-tuning complete! Model ID:", ft_model)


In [ ]:
# =============================================================
# Step 5 — Evaluate Simulated Fine-Tuned Model
# =============================================================
import numpy as np
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import re

print("\n🧮 Evaluating simulated fine-tuned model performance...")

# Use small sample of validation data
val_subset = val_df.sample(min(20, len(val_df)), random_state=42).reset_index(drop=True)
prompts = val_subset["prompt"].tolist()
actuals = val_subset["completion"].tolist()

# Convert actuals into numeric form (if applicable)
def extract_number(x):
    match = re.findall(r"[-+]?\d*\.?\d+", str(x))
    return float(match[0]) if match else np.random.uniform(70, 90)

actual_values = [extract_number(a) for a in actuals]

# 🧪 Simulate predicted values (normally would come from API)
predicted_values = [v + np.random.uniform(-3, 3) for v in actual_values]

# Calculate Mean Absolute Error
mae = mean_absolute_error(actual_values, predicted_values)
print(f"\n📊 Validation Mean Absolute Error (Simulated): {mae:.2f}")

# Plot comparison
plt.figure(figsize=(6, 4))
plt.plot(predicted_values, label="Predicted", marker="o")
plt.plot(actual_values, label="Actual", marker="x")
plt.title("Validation Predictions vs Actuals (Simulated)")
plt.xlabel("Sample Index")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()

# Reflection Summary
print("\n===== WEEK 6 REFLECTION =====")
print("✅ Completed full fine-tuning workflow (simulated) successfully.")
print("🧠 Understood how fine-tuning integrates with GPT-4o-mini API workflow.")
print(f"📊 Validation MAE (simulated): {mae:.2f}")
print("🔍 Practiced prompt alignment, data curation, and evaluation safely.")
print("💡 Next step: Try real fine-tuning (simulate=False) on small data if credits are available.")
